# 고전적인 기법으로 추출하는 차선 추적

In [ ]:
import cv2
import numpy as np
import logging
import math
import datetime
import sys

_SHOW_IMAGE = False  # 이미지를 표시할지 여부를 설정하는 플래그

In [ ]:
def detect_lane(frame):
    # Lane Detection 단계에서 사용하는 코드 
    # 입력은 비디오 프레임, 출력은 감지된 차선 라인의 정보과 시각화된 이미지이다. 
    
    logging.debug('detecting lane lines...')

    # 1. 엣지 감지 수행
    edges = detect_edges(frame)
    show_image('edges', edges)

    # 2. 관심 영역(ROI)을 설정하여 도로 부분만 유지
    cropped_edges = region_of_interest(edges)
    show_image('edges cropped', cropped_edges)

    # 3. 허프 변환을 통해 선분 감지
    line_segments = detect_line_segments(cropped_edges)
    line_segment_image = display_lines(frame, line_segments)
    show_image("line segments", line_segment_image)

    # 4. 감지된 선분을 평균화하여 차선 생성
    lane_lines = average_slope_intercept(frame, line_segments)
    lane_lines_image = display_lines(frame, lane_lines)
    show_image("lane lines", lane_lines_image)

    return lane_lines, lane_lines_image


In [ ]:
class HandCodedLaneFollower(object):

    def __init__(self, car=None):
        # 클래스 생성자: 차선 추적기를 초기화
        # car: 연결된 자동차 객체. 없으면 None으로 설정
        logging.info('Creating a HandCodedLaneFollower...')
        self.car = car  # 자동차 객체 (None일 수 있음)
        self.curr_steering_angle = 90  # 초기 조향각은 90도로 설정 (직진)

    def follow_lane(self, frame):
        # 차선 추적의 주요 함수. 입력된 프레임에서 차선을 탐지하고 조향각을 조정.
        show_image("orig", frame)  # 원본 이미지를 보여줌 (디버깅용)

        # 차선 감지 수행
        lane_lines, frame = detect_lane(frame)

        # 감지된 차선을 기반으로 조향 수행
        final_frame = self.steer(frame, lane_lines)
        return final_frame

    def steer(self, frame, lane_lines):
        logging.debug('steering...')
        if len(lane_lines) == 0:
            # 차선을 감지하지 못한 경우
            logging.error('No lane lines detected, nothing to do.')
            return frame  # 조향 없이 원본 프레임 반환

        # 차선 데이터를 사용하여 새 조향각 계산
        new_steering_angle = compute_steering_angle(frame, lane_lines)

        # 현재 조향각을 새로 계산된 조향각으로 안정화
        self.curr_steering_angle = stabilize_steering_angle(
            self.curr_steering_angle, new_steering_angle, len(lane_lines)
        )

        # 자동차 객체가 있을 경우 조향각 설정
        if self.car is not None:
            self.car.front_wheels.turn(self.curr_steering_angle)

        # 계산된 조향각을 시각적으로 보여주는 이미지 생성
        curr_heading_image = display_heading_line(frame, self.curr_steering_angle)
        show_image("heading", curr_heading_image)  # 조향각이 표시된 이미지를 보여줌
        return curr_heading_image
